# **1225. Report Contiguous Dates**
``` console
Table: Failed

+--------------+---------+
| Column Name  | Type    |
+--------------+---------+
| fail_date    | date    |
+--------------+---------+
fail_date is the primary key (column with unique values) for this table.
This table contains the days of failed tasks.
 

Table: Succeeded

+--------------+---------+
| Column Name  | Type    |
+--------------+---------+
| success_date | date    |
+--------------+---------+
success_date is the primary key (column with unique values) for this table.
This table contains the days of succeeded tasks.
 

A system is running one task every day. Every task is independent of the previous tasks. The tasks can fail or succeed.

Write a solution to report the period_state for each continuous interval of days in the period from 2019-01-01 to 2019-12-31.

period_state is 'failed' if tasks in this interval failed or 'succeeded' if tasks in this interval succeeded. Interval of days are retrieved as start_date and end_date.

Return the result table ordered by start_date.

The result format is in the following example.

 

Example 1:

Input: 
Failed table:
+-------------------+
| fail_date         |
+-------------------+
| 2018-12-28        |
| 2018-12-29        |
| 2019-01-04        |
| 2019-01-05        |
+-------------------+

Succeeded table:
+-------------------+
| success_date      |
+-------------------+
| 2018-12-30        |
| 2018-12-31        |
| 2019-01-01        |
| 2019-01-02        |
| 2019-01-03        |
| 2019-01-06        |
+-------------------+

Output: 
+--------------+--------------+--------------+
| period_state | start_date   | end_date     |
+--------------+--------------+--------------+
| succeeded    | 2019-01-01   | 2019-01-03   |
| failed       | 2019-01-04   | 2019-01-05   |
| succeeded    | 2019-01-06   | 2019-01-06   |
+--------------+--------------+--------------+

Explanation: 
The report ignored the system state in 2018 as we care about the system in the period 2019-01-01 to 2019-12-31.
From 2019-01-01 to 2019-01-03 all tasks succeeded and the system state was "succeeded".
From 2019-01-04 to 2019-01-05 all tasks failed and the system state was "failed".
From 2019-01-06 to 2019-01-06 all tasks succeeded and the system state was "succeeded".
```

**Solution-1:**

``` sql
# Write your MySQL query statement below

SELECT 
period_state, MIN(date) AS start_date, MAX(date) as end_date
FROM
(SELECT *,
DATE_ADD(date, INTERVAL -ROW_NUMBER() OVER (PARTITION BY period_state ORDER BY date) DAY) AS group_id
FROM
(SELECT 'succeeded' AS period_state, success_date as date FROM Succeeded WHERE YEAR(success_date) = 2019
UNION ALL
SELECT 'failed' AS period_state, fail_date as date FROM Failed WHERE YEAR(fail_date) = 2019)a)b
GROUP BY period_state,group_id
ORDER BY start_date

**Solution-2:**

``` sql
# Trick is all the contiguous days will have the same date when we subtract from its corresponding rank

SELECT 'failed' AS period_state, MIN(fail_date) AS start_date, MAX(fail_date) AS end_date
FROM (SELECT fail_date, ROW_NUMBER()OVER() AS rnk
     FROM Failed
     WHERE YEAR(fail_date)=2019)a
GROUP BY DATE_SUB(fail_date,INTERVAL rnk day)

UNION 

SELECT 'succeeded' AS period_state, MIN(success_date) AS start_date, MAX(success_date) AS end_date
FROM (SELECT success_date, ROW_NUMBER() OVER() AS rnk
     FROM Succeeded
     WHERE YEAR(success_date)=2019)b
GROUP BY DATE_SUB(success_date, INTERVAL rnk day)

ORDER BY start_date;